In [1]:
# populate growbal link for service provider profiles

import os
import django
import sys
import mysql.connector as mc
from datetime import datetime
from mysql.connector.errors import IntegrityError
import asyncio
from dotenv import load_dotenv

# Load environment variables from 1.env file
load_dotenv("envs/1.env")

# Setup Django
sys.path.insert(0, 'growbal_django')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "growbal.settings")
django.setup()

from accounts.models import ServiceProviderProfile
from accounts.serializers import ServiceProviderProfileSerializer
from asgiref.sync import sync_to_async


@sync_to_async
def get_all_service_provider_profiles_with_websites():
    """Get all ServiceProviderProfile instances that have a website field"""
    profiles = ServiceProviderProfile.objects.filter(website__isnull=False).exclude(website='')
    return list(profiles.values('id', 'name', 'website', 'growbal_link'))


@sync_to_async
def update_growbal_link(profile_id, growbal_link):
    """Update the growbal_link field for a specific ServiceProviderProfile"""
    try:
        profile = ServiceProviderProfile.objects.get(id=profile_id)
        profile.growbal_link = growbal_link
        profile.save()
        return True
    except ServiceProviderProfile.DoesNotExist:
        print(f"Profile with ID {profile_id} not found")
        return False


def normalize_website_url(url):
    """Normalize website URL for comparison"""
    if not url:
        return None
    
    # Remove protocol and trailing slashes
    url = url.lower().strip()
    if url.startswith('http://'):
        url = url[7:]
    elif url.startswith('https://'):
        url = url[8:]
    
    # Remove www. prefix
    if url.startswith('www.'):
        url = url[4:]
    
    # Remove trailing slash
    url = url.rstrip('/')
    
    return url


def find_matching_establishment(django_website, mysql_cursor, mysql_schema):
    """Find a matching establishment in MySQL based on website"""
    if not django_website:
        return None
    
    normalized_django_website = normalize_website_url(django_website)
    if not normalized_django_website:
        return None
    
    # Query MySQL for establishments with matching website
    query = f"SELECT id, company_name, web_site, slug FROM `{mysql_schema}`.`establishment` WHERE web_site IS NOT NULL AND web_site != ''"
    mysql_cursor.execute(query)
    establishments = mysql_cursor.fetchall()
    
    for establishment in establishments:
        mysql_website = establishment.get('web_site')
        normalized_mysql_website = normalize_website_url(mysql_website)
        
        if normalized_mysql_website and normalized_django_website == normalized_mysql_website:
            return establishment
    
    return None


async def populate_growbal_links():
    # Load MySQL credentials from environment
    MYSQL_HOST = os.getenv("MYSQL_AUTH_HOST")
    MYSQL_PORT = int(os.getenv("MYSQL_AUTH_PORT", 3306))
    MYSQL_USER = os.getenv("MYSQL_AUTH_USERNAME")
    MYSQL_PASSWORD = os.getenv("MYSQL_AUTH_PASSWORD")
    MYSQL_SCHEMA = os.getenv("MYSQL_AUTH_DATABASE")
    
    print("Loading credentials from 1.env file...")
    print(f"MySQL Host: {MYSQL_HOST}")
    print(f"MySQL Port: {MYSQL_PORT}")
    print(f"MySQL Database: {MYSQL_SCHEMA}")
    
    # Get all Django profiles with websites
    print("Fetching ServiceProviderProfile records with websites...")
    django_profiles = await get_all_service_provider_profiles_with_websites()
    print(f"Found {len(django_profiles)} profiles with websites")
    
    if not django_profiles:
        print("No profiles with websites found. Exiting.")
        return
    
    # Connect to MySQL
    print("Connecting to MySQL...")
    try:
        cnx = mc.connect(
            host=MYSQL_HOST,
            port=MYSQL_PORT,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
            database=MYSQL_SCHEMA,
            auth_plugin="mysql_native_password"
        )
        cur = cnx.cursor(dictionary=True)
        print("Successfully connected to MySQL")
    except Exception as e:
        print(f"Failed to connect to MySQL: {e}")
        return
    
    matched_count = 0
    updated_count = 0
    skipped_count = 0
    
    try:
        print("\nStarting website matching process...")
        
        for profile in django_profiles:
            profile_id = profile['id']
            profile_name = profile['name']
            django_website = profile['website']
            current_growbal_link = profile['growbal_link']
            
            print(f"\nProcessing profile: {profile_name} (ID: {profile_id})")
            print(f"Django website: {django_website}")
            
            # Find matching establishment in MySQL
            matching_establishment = find_matching_establishment(django_website, cur, MYSQL_SCHEMA)
            
            if matching_establishment:
                matched_count += 1
                mysql_slug = matching_establishment['slug']
                mysql_company = matching_establishment['company_name']
                mysql_website = matching_establishment['web_site']
                
                print(f"✓ Found match in MySQL:")
                print(f"  Company: {mysql_company}")
                print(f"  MySQL website: {mysql_website}")
                print(f"  Slug: {mysql_slug}")
                
                # Generate new growbal_link
                new_growbal_link = f"https://staging.growbal.net/agent/{mysql_slug}"
                
                if current_growbal_link != new_growbal_link:
                    print(f"  Updating growbal_link: {new_growbal_link}")
                    
                    # Update the Django profile
                    success = await update_growbal_link(profile_id, new_growbal_link)
                    if success:
                        updated_count += 1
                        print(f"  ✓ Successfully updated growbal_link")
                    else:
                        print(f"  ✗ Failed to update growbal_link")
                else:
                    skipped_count += 1
                    print(f"  → Growbal_link already up to date")
            else:
                print(f"✗ No matching establishment found in MySQL")
        
        print(f"\n" + "="*60)
        print(f"SUMMARY:")
        print(f"Total profiles processed: {len(django_profiles)}")
        print(f"Matches found: {matched_count}")
        print(f"Records updated: {updated_count}")
        print(f"Records skipped (already up to date): {skipped_count}")
        print(f"No matches found: {len(django_profiles) - matched_count}")
        
    except Exception as e:
        print(f"Error during processing: {e}")
        raise e
    
    finally:
        cur.close()
        cnx.close()
        print("MySQL connection closed")

# Run the population function
await populate_growbal_links()

Loading credentials from 1.env file...
MySQL Host: 16.16.162.51
MySQL Port: 3306
MySQL Database: growbal
Fetching ServiceProviderProfile records with websites...
Found 482 profiles with websites
Connecting to MySQL...
Successfully connected to MySQL

Starting website matching process...

Processing profile: PAAY & Co LLP (ID: 426)
Django website: https://www.paay.co.in
✓ Found match in MySQL:
  Company: PAAY & Co LLP
  MySQL website: https://www.paay.co.in
  Slug: lr9O3ooq
  → Growbal_link already up to date

Processing profile: AEY Group (ID: 41)
Django website: https://aey.ae
✓ Found match in MySQL:
  Company: AEY Group
  MySQL website: https://aey.ae
  Slug: 1TXAAfPX
  → Growbal_link already up to date

Processing profile: AKA Management Consultancy (ID: 42)
Django website: https://aka.ae
✓ Found match in MySQL:
  Company: AKA Management Consultancy
  MySQL website: https://aka.ae
  Slug: jAjI9UWY
  → Growbal_link already up to date

Processing profile: A & H Consultants LLC (ID: 43